In [55]:
!pip install polars autopep8
import polars as pl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 732.0 kB/s eta 0:00:00a 0:00:01


#　前処理大全

## 13-1

In [23]:
df_customer = pl.read_csv('./data/customer.csv')
df_hotel = pl.read_csv('./data/hotel.csv')
df_reserve = pl.read_csv('./data/reserve.csv')

In [115]:
df_joined = df_reserve.join(df_hotel, on='hotel_id').join(df_customer, on='customer_id')
df_joined

reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business,age,sex,home_latitude,home_longitude
str,str,str,str,str,str,str,i64,i64,i64,str,str,f64,f64,bool,i64,str,f64,f64
"""r1""","""h_75""","""c_1""","""2016-03-06 13:…","""2016-03-26""","""10:00:00""","""2016-03-29""",4,97200,8100,"""B""","""B-2""",35.54586,139.701217,false,41,"""man""",35.092193,136.512347
"""r2""","""h_219""","""c_1""","""2016-07-16 23:…","""2016-07-20""","""11:30:00""","""2016-07-21""",2,20600,10300,"""B""","""B-3""",35.644729,139.693389,true,41,"""man""",35.092193,136.512347
"""r3""","""h_179""","""c_1""","""2016-09-24 10:…","""2016-10-19""","""09:00:00""","""2016-10-22""",2,33600,5600,"""G""","""G-4""",33.599962,130.632019,false,41,"""man""",35.092193,136.512347
"""r4""","""h_214""","""c_1""","""2017-03-08 03:…","""2017-03-29""","""11:00:00""","""2017-03-30""",4,194400,48600,"""C""","""C-2""",38.333994,140.791836,false,41,"""man""",35.092193,136.512347
"""r5""","""h_16""","""c_1""","""2017-09-05 19:…","""2017-09-22""","""10:30:00""","""2017-09-23""",3,68100,22700,"""A""","""A-3""",35.911393,139.932511,false,41,"""man""",35.092193,136.512347
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""r4026""","""h_129""","""c_999""","""2017-06-27 23:…","""2017-07-10""","""09:30:00""","""2017-07-11""",2,16000,8000,"""D""","""D-2""",43.059919,141.510445,false,48,"""woman""",38.1728,140.464198
"""r4027""","""h_97""","""c_999""","""2017-09-29 05:…","""2017-10-09""","""10:30:00""","""2017-10-10""",2,41800,20900,"""C""","""C-2""",38.336324,140.696792,false,48,"""woman""",38.1728,140.464198
"""r4028""","""h_27""","""c_999""","""2018-03-14 05:…","""2018-04-02""","""11:30:00""","""2018-04-04""",2,74800,18700,"""D""","""D-1""",43.057724,141.412532,false,48,"""woman""",38.1728,140.464198


In [124]:
df_joined['total_price'].median()

64800.0

In [164]:
df_joined_with_age_group = df_joined.with_columns(
    pl.when( pl.col('age') <= 60 )
    .then( pl.col('age') // 10 )
    .otherwise( 7 )
    .alias('age_group')
)

total_stat = df_joined_with_age_group.group_by('age_group').agg([
    ((pl.col('age_group').mean()*10).cast(pl.Int16).cast(pl.String) + '代').alias('年齢グループ'),
    pl.col('people_num').sum().alias('該当人数'),
    pl.col('age_group').len().alias('合計予約回数'),
    pl.col('people_num').mean().alias('平均予約人数'),
    (pl.col('total_price').sum() / pl.col('total_price').len()).alias('平均予約単価'),
])

result = total_stat.sort('年齢グループ').drop('age_group')

result

年齢グループ,該当人数,合計予約回数,平均予約人数,平均予約単価
str,i64,u32,f64,f64
"""20代""",226,87,2.597701,121602.298851
"""30代""",2522,978,2.578732,104611.042945
"""40代""",2541,1009,2.518335,101928.444004
"""50代""",1599,638,2.50627,100815.047022
"""60代""",174,77,2.25974,95094.805195
"""70代""",3183,1241,2.564867,103125.463336
